<a href="https://colab.research.google.com/github/Zoro1092000/Do_an_tot_nghiep/blob/main/Tangcuongdulieu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py
import torch
import numpy as np
import collections
import random

In [2]:
def write_single_graph(graph_id, x, edge_index, y, edge_y):
    '''
    store into hdf5 file
    '''
    with h5py.File("/content/kadem_train.hdf5", 'a') as f:
      f.create_dataset(f'{graph_id}/x', data=x, dtype = 'float32')
      f.create_dataset(f'{graph_id}/edge_index', data=edge_index, dtype = 'int64')
      f.create_dataset(f'{graph_id}/y', data=y, dtype = 'uint8')
      f.create_dataset(f'{graph_id}/edge_y', data=edge_y, dtype = 'uint8')

In [3]:
num_train_graph = 768
num_val_graph = 96
num_test_graph = 96
with h5py.File("/content/drive/Shareddrives/botnetdata/Kademlia/processed/kadem_train.hdf5", 'r') as f:

  for graph_id in range(num_train_graph):
    print(f[f'{graph_id}'])
    num_node = f[f'{graph_id}']['x'].shape[0]
    edge_index = f[f'{graph_id}']['edge_index']
    label = f[f'{graph_id}']['y']
    edge_y = f[f'{graph_id}']['edge_y']
    x = f[f'{graph_id}']['x'][:]

    list_node = f[f'{graph_id}']['y'][:]
    # print(f"list_node_shape: {list_node.shape}")
    list_botnode = np.array([])
    for i in range(num_node):
      if list_node[i] == 1:
        list_botnode = np.append(list_botnode, i)

    # print(f"list_botnode_shape: {list_botnode.shape}")
    # print(list_botnode)

    list_edge = f[f'{graph_id}']['edge_index'][:]
    list_label_edge = f[f'{graph_id}']['edge_y'][:]
    print(list_edge.shape[1])
    print(list_label_edge.shape)
    list_botedge = None
    list_source_botnode = np.array([])
    list_destination_botnode = np.array([])

    for i in range(list_edge.shape[1]):
      if list_label_edge[i] == 1:
        list_source_botnode = np.append(list_source_botnode, list_edge[0][i])
        list_destination_botnode = np.append(list_destination_botnode, list_edge[1][i])

    list_botedge = np.array([list_source_botnode, list_destination_botnode])
    # print(list_edge.shape)
    # print(list_source_botnode.shape)
    # print(list_botedge.shape)
    # print(list_botedge)
    
    random_list_node = np.array([])
    n = -1
    for i in range(list_botnode.shape[0]):
      while n == -1 or list_node[n] == 1 or n in random_list_node:
        n = np.random.randint(num_node)
      random_list_node = np.append(random_list_node, n)

    print(random_list_node.shape)
    print(list_botnode.shape)

    dict_map = {}
    for i in range(list_botnode.shape[0]):
      dict_map[list_botnode[i]] = random_list_node[i]

    new_list_botedge = np.vectorize(dict_map.get)(list_botedge)

    new_label = label[:]
    for i in random_list_node:
      new_label[i.astype(int)] = 1

    edge_index = edge_index[:]
    print(edge_index.shape)
    
    cache = np.ones(new_list_botedge.shape[1])
    list_label_edge = np.append(list_label_edge, cache)

    print("------------")
    print(new_list_botedge[0].shape)
    new_source_node = np.append(edge_index[0], new_list_botedge[0])
    new_destination_node = np.append(edge_index[1], new_list_botedge[1])
    new_edge_index = np.array([new_source_node, new_destination_node])
    print(new_edge_index.shape)

    # print("Check before write to file")
    # print(label.shape)
    # print(type(label))
    # print(new_label.shape)
    # print(type(new_label))
    # print("----------")
    # print(type(edge_index))
    # print(edge_index.shape)
    # print(type(new_edge_index))
    # print(new_edge_index.shape)
    print("+++++++++++++")
    print(list_label_edge.shape)
    write_single_graph(graph_id=graph_id, x=x, edge_index=new_edge_index, y=new_label, edge_y=list_label_edge)

Streaming output truncated to the last 5000 lines.
(2, 1536692)
------------
(60029,)
(2, 1596721)
+++++++++++++
(1596721,)
<HDF5 group "/314" (4 members)>
1535302
(1535302,)
(10000,)
(10000,)
(2, 1535302)
------------
(60254,)
(2, 1595556)
+++++++++++++
(1595556,)
<HDF5 group "/315" (4 members)>
1475817
(1475817,)
(10000,)
(10000,)
(2, 1475817)
------------
(60025,)
(2, 1535842)
+++++++++++++
(1535842,)
<HDF5 group "/316" (4 members)>
1538335
(1538335,)
(10000,)
(10000,)
(2, 1538335)
------------
(60154,)
(2, 1598489)
+++++++++++++
(1598489,)
<HDF5 group "/317" (4 members)>
1521572
(1521572,)
(10000,)
(10000,)
(2, 1521572)
------------
(60605,)
(2, 1582177)
+++++++++++++
(1582177,)
<HDF5 group "/318" (4 members)>
1516207
(1516207,)
(10000,)
(10000,)
(2, 1516207)
------------
(59818,)
(2, 1576025)
+++++++++++++
(1576025,)
<HDF5 group "/319" (4 members)>
1509626
(1509626,)
(10000,)
(10000,)
(2, 1509626)
------------
(60305,)
(2, 1569931)
+++++++++++++
(1569931,)
<HDF5 group "/320" (4 me

In [4]:
!cp -r /content/kadem_train.hdf5 /content/drive/Shareddrives/botnetdata/kadem_20k_botnode/processed